## 🔧 Library Imports

We begin by importing essential libraries required for training a CycleGAN using PyTorch. These include modules for neural network construction (`torch.nn`), optimization (`torch.optim`), image preprocessing (`torchvision.transforms`), dataset loading, and image manipulation via PIL. We also import helper modules for dynamic computation graphs and data iteration.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import os
import itertools
from torch.autograd import Variable

## 🧹 Clear CUDA Cache

Before training begins, we clear the CUDA memory cache to prevent potential memory issues on the GPU.


In [ ]:
# Clear CUDA cache
torch.cuda.empty_cache()

## 💻 Select Compute Device

We define the computation device, defaulting to GPU (`cuda`) if available, otherwise falling back to CPU. This enables seamless training on Colab’s hardware accelerators.


In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

## 📁 Mount Google Drive

To access the dataset stored in Google Drive, we mount the drive into the Colab runtime. This allows for persistent access to image folders and model checkpoints.


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

## 🖼️ Load and Preprocess Image Data

We define a preprocessing pipeline using `torchvision.transforms` to resize and normalize the images. We then load the benign and malignant training images using `ImageFolder`, and wrap them in `DataLoader` objects for efficient batch iteration during training.


In [ ]:
#Load Data
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset_A = datasets.ImageFolder('/content/drive/MyDrive/my_data/train/benign', transform=transform)
dataset_B = datasets.ImageFolder('/content/drive/MyDrive/my_data/train/malignant', transform=transform)

loader_A = DataLoader(dataset_A, batch_size=1, shuffle=True)
loader_B = DataLoader(dataset_B, batch_size=1, shuffle=True)

## 🧪 Visualize Sample Images

To verify the correctness of the loaded data, we visualize one sample image each from the benign and malignant datasets. This step helps confirm that data loading and transformations are working as intended.


In [ ]:
import matplotlib.pyplot as plt

def imshow(img):
    img = img.numpy().transpose((1, 2, 0))
    plt.imshow(img)
    plt.axis('off')

dataiter_A = iter(loader_A)
images_A, _ = next(dataiter_A)

dataiter_B = iter(loader_B)
images_B, _ = next(dataiter_B)

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.title('Sample from trainA (benign)')
imshow(images_A[0])

plt.subplot(1, 2, 2)
plt.title('Sample from trainB (malignant)')
imshow(images_B[0])
plt.show()

## ⚙️ Hyperparameters

We define key hyperparameters for model training, including input/output channels, image size, learning rate, and the number of residual blocks in the generator architecture.


In [ ]:
# Parameters
input_nc = 3
output_nc = 3
size = 256
#batch_size = 50
lr = 0.0002
beta1 = 0.5
n_residual_blocks = 9

## 🧠 Define Generator Architecture with Residual Blocks

Here we define the CycleGAN Generator model using a residual architecture. The generator includes:

- An initial convolution block,
- Downsampling layers,
- Multiple residual blocks,
- Upsampling layers,
- An output layer with Tanh activation.

Residual connections help retain feature integrity while transforming images across domains.


In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features)
        )

    def forward(self, x):
        return x + self.block(x)

class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, n_residual_blocks=9):
        super(Generator, self).__init__()

        # Initial convolution block
        model = [nn.ReflectionPad2d(3),
                 nn.Conv2d(input_nc, 64, 7),
                 nn.InstanceNorm2d(64),
                 nn.ReLU(inplace=True)]

        # Downsampling
        in_features = 64
        out_features = in_features*2
        for _ in range(2):
            model += [nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                      nn.InstanceNorm2d(out_features),
                      nn.ReLU(inplace=True)]
            in_features = out_features
            out_features = in_features*2

        # Residual blocks
        for _ in range(n_residual_blocks):
            model += [ResidualBlock(in_features)]

        # Upsampling
        out_features = in_features//2
        for _ in range(2):
            model += [nn.ConvTranspose2d(in_features, out_features, 3, stride=2, padding=1, output_padding=1),
                      nn.InstanceNorm2d(out_features),
                      nn.ReLU(inplace=True)]
            in_features = out_features
            out_features = in_features//2

        # Output layer
        model += [nn.ReflectionPad2d(3),
                  nn.Conv2d(64, output_nc, 7),
                  nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)


## 🕵️ Define Discriminator Architecture

We implement the PatchGAN discriminator, which evaluates the realism of image patches instead of full images. It consists of stacked convolutional layers with increasing depth, enabling the model to distinguish between real and translated images.


In [ ]:
class Discriminator(nn.Module):
    def __init__(self, input_nc):
        super(Discriminator, self).__init__()

        model = [nn.Conv2d(input_nc, 64, 4, stride=2, padding=1),
                 nn.LeakyReLU(0.2, inplace=True)]

        model += [nn.Conv2d(64, 128, 4, stride=2, padding=1),
                  nn.InstanceNorm2d(128),
                  nn.LeakyReLU(0.2, inplace=True)]

        model += [nn.Conv2d(128, 256, 4, stride=2, padding=1),
                  nn.InstanceNorm2d(256),
                  nn.LeakyReLU(0.2, inplace=True)]

        model += [nn.Conv2d(256, 512, 4, padding=1),
                  nn.InstanceNorm2d(512),
                  nn.LeakyReLU(0.2, inplace=True)]

        # Output layer
        model += [nn.Conv2d(512, 1, 4, padding=1)]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

## 🛠️ Initialize Generator and Discriminator Models

We instantiate two generators (A→B and B→A) and two discriminators (one for each domain), moving all models to the selected compute device (CPU or GPU).


In [ ]:
# Initialize models
netG_A2B = Generator(input_nc, output_nc, n_residual_blocks).to(device)
netG_B2A = Generator(input_nc, output_nc, n_residual_blocks).to(device)
netD_A = Discriminator(input_nc).to(device)
netD_B = Discriminator(input_nc).to(device)

## 🚀 Define Optimizers and Loss Functions

We set up separate Adam optimizers for the generators and discriminators. The losses used are:

- **MSE Loss** for adversarial learning,
- **L1 Loss** for cycle-consistency and identity constraints,
- **CrossEntropy Loss** (prepared for optional classification tasks).


In [ ]:
# Optimizers
optimizer_G = optim.Adam(itertools.chain(netG_A2B.parameters(), netG_B2A.parameters()), lr=lr, betas=(beta1, 0.999))
optimizer_D_A = optim.Adam(netD_A.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_D_B = optim.Adam(netD_B.parameters(), lr=lr, betas=(beta1, 0.999))

# Define loss functions
criterion_GAN = nn.MSELoss().to(device)
criterion_cycle = nn.L1Loss().to(device)
criterion_identity = nn.L1Loss().to(device)
criterion_classification = nn.CrossEntropyLoss().to(device)


## 🖼️ Utility: Visualize Translated Images

This helper function allows us to visualize real and fake images during training. It shows side-by-side comparisons between real images from both domains and their translated counterparts.


In [ ]:
import matplotlib.pyplot as plt

import os

def visualize_generated_images(real_A, real_B, fake_A, fake_B):
    """Visualize one sample translation from both generators."""
    with torch.no_grad():
        # Assuming normalization was used
        real_A = 0.5 * (real_A + 1.0)
        real_B = 0.5 * (real_B + 1.0)
        fake_A = 0.5 * (fake_A + 1.0)
        fake_B = 0.5 * (fake_B + 1.0)

        plt.figure(figsize=(8, 8))
        plt.subplot(221)
        plt.title("Real A")
        plt.imshow(real_A[0].cpu().permute(1, 2, 0))
        plt.axis('off')

        plt.subplot(222)
        plt.title("Fake B")
        plt.imshow(fake_B[0].cpu().permute(1, 2, 0))
        plt.axis('off')

        plt.subplot(223)
        plt.title("Real B")
        plt.imshow(real_B[0].cpu().permute(1, 2, 0))
        plt.axis('off')

        plt.subplot(224)
        plt.title("Fake A")
        plt.imshow(fake_A[0].cpu().permute(1, 2, 0))
        plt.axis('off')

        plt.show()

## 💾 Save Model Checkpoints to Google Drive

To preserve training progress, this function saves the state dictionaries of all generator and discriminator models after each epoch to a specified directory in Google Drive.


In [ ]:
import os
import torch

def save_models_to_drive(epoch, netG_A2B, netG_B2A, netD_A, netD_B, drive_path='/content/drive/MyDrive/CycleGAN_Models'):
    """
    Save model parameters to Google Drive.

    Parameters:
        epoch (int): The current epoch number.
        netG_A2B (nn.Module): Generator model from domain A to B.
        netG_B2A (nn.Module): Generator model from domain B to A.
        netD_A (nn.Module): Discriminator model for domain A.
        netD_B (nn.Module): Discriminator model for domain B.
        drive_path (str): The path in Google Drive to save the models.
    """
    if not os.path.exists(drive_path):
        os.makedirs(drive_path)

    # Define file paths for saving
    path_G_A2B = os.path.join(drive_path, f'netG_A2B_epoch_{epoch}.pth')
    path_G_B2A = os.path.join(drive_path, f'netG_B2A_epoch_{epoch}.pth')
    path_D_A = os.path.join(drive_path, f'netD_A_epoch_{epoch}.pth')
    path_D_B = os.path.join(drive_path, f'netD_B_epoch_{epoch}.pth')

    # Save the models
    torch.save(netG_A2B.state_dict(), path_G_A2B)
    torch.save(netG_B2A.state_dict(), path_G_B2A)
    torch.save(netD_A.state_dict(), path_D_A)
    torch.save(netD_B.state_dict(), path_D_B)

    print(f"Saved models at epoch {epoch} to {drive_path}")

# Example usage within the training loop:
# save_models_to_drive(epoch, netG_A2B, netG_B2A, netD_A, netD_B)


In [ ]:
# Clear CUDA cache
torch.cuda.empty_cache()

## 🎯 Training Loop: CycleGAN for Domain Translation

We now train the CycleGAN model for a specified number of epochs. The training process includes:

- **Identity Loss**: Preserves features when input and output domains are the same.
- **Adversarial Loss**: Guides the generators to produce realistic images.
- **Cycle-consistency Loss**: Ensures image transformation is reversible.
- **Discriminator Updates**: Enables learning to distinguish real from generated images.

We also include real-time visualization and periodic saving of model checkpoints to Google Drive. Training durations are tracked per epoch and in total.


In [ ]:
import torch
import itertools
from torch.autograd import Variable
import time

# Parameters
num_epochs = 25
lambda_id = 0.1
lambda_cycle = 10

# Record the total training start time
total_training_start_time = time.time()


# Path to save images in Google Drive
#drive_image_path = '/content/drive/My Drive/CycleGAN_Images'

for epoch in range(num_epochs):

    # Record the start time of the epoch
    epoch_start_time = time.time()

    for i, (data_A, data_B) in enumerate(zip(loader_A, loader_B)):
        real_A = Variable(data_A[0].to(device))
        real_B = Variable(data_B[0].to(device))

        # ----------------------
        #  Train Generators A2B and B2A
        # ----------------------
        optimizer_G.zero_grad()

        # Identity loss
        same_B = netG_A2B(real_B)
        loss_identity_B = criterion_identity(same_B, real_B) * lambda_cycle * lambda_id
        same_A = netG_B2A(real_A)
        loss_identity_A = criterion_identity(same_A, real_A) * lambda_cycle * lambda_id

        # GAN loss
        fake_B = netG_A2B(real_A)
        pred_fake = netD_B(fake_B)
        loss_GAN_A2B = criterion_GAN(pred_fake, Variable(torch.ones(pred_fake.size()).to(device)))

        fake_A = netG_B2A(real_B)
        pred_fake = netD_A(fake_A)
        loss_GAN_B2A = criterion_GAN(pred_fake, Variable(torch.ones(pred_fake.size()).to(device)))

        # Cycle loss
        recovered_A = netG_B2A(fake_B)
        loss_cycle_ABA = criterion_cycle(recovered_A, real_A) * lambda_cycle

        recovered_B = netG_A2B(fake_A)
        loss_cycle_BAB = criterion_cycle(recovered_B, real_B) * lambda_cycle

        # Total loss
        total_loss_G = loss_identity_A + loss_identity_B + loss_GAN_A2B + loss_GAN_B2A + loss_cycle_ABA + loss_cycle_BAB
        total_loss_G.backward()
        optimizer_G.step()

        # -----------------------
        #  Train Discriminator D_A
        # -----------------------
        optimizer_D_A.zero_grad()

        # Real loss
        pred_real = netD_A(real_A)
        loss_D_real = criterion_GAN(pred_real, Variable(torch.ones(pred_real.size()).to(device)))

        # Fake loss
        pred_fake = netD_A(fake_A.detach())
        loss_D_fake = criterion_GAN(pred_fake, Variable(torch.zeros(pred_fake.size()).to(device)))

        # Total loss
        loss_D_A = (loss_D_real + loss_D_fake) * 0.5
        loss_D_A.backward()
        optimizer_D_A.step()

        # -----------------------
        #  Train Discriminator D_B
        # -----------------------
        optimizer_D_B.zero_grad()

        # Real loss
        pred_real = netD_B(real_B)
        loss_D_real = criterion_GAN(pred_real, Variable(torch.ones(pred_real.size()).to(device)))

        # Fake loss
        pred_fake = netD_B(fake_B.detach())
        loss_D_fake = criterion_GAN(pred_fake, Variable(torch.zeros(pred_fake.size()).to(device)))

        # Total loss
        loss_D_B = (loss_D_real + loss_D_fake) * 0.5
        loss_D_B.backward()
        optimizer_D_B.step()

        # Visualize generated images periodically
        if i % 100 == 0:  # Adjust this value based on your training dataset size
            visualize_generated_images(real_A, real_B, fake_A, fake_B)

        # Print training progress
        if i % 10 == 0:  # Adjust the printing frequency as needed
            print(f"Epoch {epoch}/{num_epochs} - Batch {i}/{min(len(loader_A), len(loader_B))}, "
                  f"Loss_G: {total_loss_G.item()}, Loss_D_A: {loss_D_A.item()}, Loss_D_B: {loss_D_B.item()}")

    # Record the end time of the epoch
    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time
    print(f"Epoch {epoch} completed in {epoch_duration:.2f} seconds")

# Save models at the end of each epoch or at specific intervals
if (epoch + 1) % 1 == 0:  # Every 10 epochs
    save_models_to_drive(epoch, netG_A2B, netG_B2A, netD_A, netD_B)


# Record the total training end time
total_training_end_time = time.time()
total_training_duration = total_training_end_time - total_training_start_time
print(f"Total training time: {total_training_duration:.2f} seconds")

    # Optionally, you can also visualize at the end of each epoch to see the progress
    #visualize_generated_images(real_A, real_B, fake_A, fake_B)

    # Evaluate the model after each epoch
    #evaluation_results = evaluate_model(netG_A2B, netG_B2A, loader_A, loader_B, device)

    # Print evaluation results
    #print(f"Epoch {epoch + 1} Evaluation Results: {evaluation_results}")


# Save generated images to Google Drive every 30 epochs
#if (epoch + 1) % 30 == 0:
      #epoch_dir = os.path.join(drive_image_path, f'epoch_{epoch + 1}')
      #save_image_to_drive(fake_A, epoch_dir, f'fake_A_epoch_{epoch + 1}.png')
      #save_image_to_drive(fake_B, epoch_dir, f'fake_B_epoch_{epoch + 1}.png')
      #save_image_to_drive(real_A, epoch_dir, f'real_A_epoch_{epoch + 1}.png')
      #save_image_to_drive(real_B, epoch_dir, f'real_B_epoch_{epoch + 1}.png')
      #save_image_to_drive(recovered_A, epoch_dir, f'recovered_A_epoch_{epoch + 1}.png')
      #save_image_to_drive(recovered_B, epoch_dir, f'recovered_B_epoch_{epoch + 1}.png')
